# Model comparison

## PLS

In [55]:
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
import numpy as np
from scipy.io import loadmat
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error, r2_score

data = loadmat('./dataset/mango/mango_dm_split.mat')
min_value, max_value = data['min_y'][-1][-1], data['max_y'][-1][-1]
retransform = lambda x: x * (max_value - min_value)

In [56]:
x_train, y_train, x_test, y_test = data['x_train'], data['y_train'], data['x_test'], data['y_test']
print(f"shape of data:\n"
      f"x_train: {x_train.shape}, y_train: {y_train.shape},\n"
      f"x_test: {x_test.shape}, y_test: {y_test.shape}")

shape of data:
x_train: (8183, 102), y_train: (8183, 1),
x_test: (3508, 102), y_test: (3508, 1)


In [57]:
pls = PLSRegression(n_components=90)
svr = SVR(kernel="rbf", degree=30, gamma="scale")
mlp = MLPRegressor(hidden_layer_sizes=(60, 50, ))
pls = pls.fit(x_train, y_train.ravel())
svr = svr.fit(x_train, y_train.ravel())
mlp = mlp.fit(x_train, y_train.ravel())

models = {'PLS': pls, "SVR": svr, "MLP": mlp}
results = {model_name: model.predict(x_test).reshape((-1, )) for model_name, model in models.items()}
for model_name, model_result in results.items():
      rmse = np.sqrt(mean_squared_error(y_test, model_result))
      print(model_name, "RMSE: ", rmse)
      print(model_name, "Dry matter content error", retransform(rmse))
      print(model_name, "R^2: ", r2_score(y_test, model_result))

PLS RMSE:  0.05722520296881164
PLS Dry matter content error 0.8648183977750965
PLS R^2:  0.8793937498230511
SVR RMSE:  0.1139650997574326
SVR Dry matter content error 1.7223025845485895
SVR R^2:  0.5216575965112935
MLP RMSE:  0.15508626630172465
MLP Dry matter content error 2.343748023280531
MLP R^2:  0.11418748397100065
